# Create a Point to Point (Layer 2) Network with Explicit Route Options (ERO)

This notebook shows how to create an isolated Point 2 Point Layer2 Ethernet, specify Explicity Route Options (ERO) to control Quality of Service and connect compute nodes to it and use FABlib's automatic configuration functionality.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Create the Experiment Slice

The following script sets up two nodes, each with dedicated NICs connected to two isolated WAN Ethernets, with and without Explicit Routes.

Two nodes are instantiated, each with a NIC component added. This script employs components modeled after NIC_ConnectX_6, representing a 100 Gpbs Mellanox ConnectX-6 PCI device. When using dedicated PCI devices, the entire physical device is allocated to one node, accessed through PCI passthrough. Using the `get_interfaces()` method on a component retrieves a list of interfaces. Many dedicated NIC components may have multiple ports, with either port connectable to the network.


NIC component model options include:
- NIC_ConnectX_5: 25 Gbps Dedicated Mellanox ConnectX-5 PCI Device (2 Ports)
- NIC_ConnectX_6: 100 Gbps Dedicated Mellanox ConnectX-6 PCI Device (2 Ports)

In [ ]:
slice_name = 'MySlice-ero'
[site1, site2, site3]  = fablib.get_random_sites(count=3)
print(f"Sites: {site1}, {site2}, {site3}")

node1_name = 'Node1'
node2_name = 'Node2'
net1_name = 'net-with-ero'
net2_name = 'net'

In [ ]:
from fim.slivers.network_service import NSLayer, ServiceType

#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=net1_name, subnet=IPv4Network("192.168.1.0/24"))

net2 = slice.add_l2network(name=net2_name, subnet=IPv4Network("192.168.2.0/24"))

# Node1
node1 = slice.add_node(name=node1_name, site=site1)
n1_nic1 = node1.add_component(model='NIC_ConnectX_5', name='nic1')
n1_nic1.get_interfaces()[0].set_mode('auto')
n1_nic1.get_interfaces()[0].set_vlan('100')
n1_nic1.get_interfaces()[1].set_mode('auto')
n1_nic1.get_interfaces()[1].set_vlan('200')

net1.add_interface(n1_nic1.get_interfaces()[0])

net2.add_interface(n1_nic1.get_interfaces()[1])

# Node2
node2 = slice.add_node(name=node2_name, site=site2)
n2_nic1 = node2.add_component(model='NIC_ConnectX_5', name='nic1')

n2_nic1.get_interfaces()[0].set_mode('auto')
n2_nic1.get_interfaces()[0].set_vlan('100')
n2_nic1.get_interfaces()[1].set_mode('auto')
n2_nic1.get_interfaces()[1].set_vlan('200')

net1.add_interface(n2_nic1.get_interfaces()[0])

net2.add_interface(n2_nic1.get_interfaces()[1])

# Set Explicit Route Options for Network1
net1.set_l2_route_hops(hops=[site3])


#Submit Slice Request
slice.submit()

## Run the Experiment
We demonstrate the variance in delay between two networks. It's important to note that the network utilizing ERO exhibits greater ping delay compared to the network without ERO.

In [ ]:
# Get Slice and Nodes
slice = fablib.get_slice(slice_name)

node1 = slice.get_node(name=node1_name)        
node2 = slice.get_node(name=node2_name)    

### Ping on the Network with Explicit Routes

In [ ]:
node2_net1_addr = node2.get_interface(network_name=net1_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_net1_addr}')

### Ping on the Network without Explicit Routes

In [ ]:
node2_net2_addr = node2.get_interface(network_name=net2_name).get_ip_addr()

stdout, stderr = node1.execute(f'ping -c 5 {node2_net2_addr}')

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice.delete()